# TensorFlow - MNIST For ML Intermediate - Next Step (Part V - Convolutional Neural Network) 

This Code Along tutorial is an extension of TensorFlow.org's A Guide to TF Layers: Building a Convolutional Neural Network tutorial. In this tutorial, we extend the five layer deep neural network (DNN), and one dropout layer, to a six layer convolutional neural network (CNN), with one convolutional layer.

In [ ]:
# Importing Tensorflow
import tensorflow as tf
from tensorflow.python.framework import ops

## What is MNIST

The MNIST dataset is a well-known dataset of images of handwritten digits (0 .. 9). It is used extensively as an example for
training beginners in both Computer Vision and ML frameworks. 

Kaggle describes this dataset as:

MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

## Images

<img src='mnist.png'>

Each image has a label, between 0 and 9 and represents a single digit. Each image is 28 pixels by 28 pixels. The pixels are black or white (0 or 1). Each image represents a total of 784 pixels (28 x 28)

## Getting the Data

Tensorflow comes with several builtin datasets to get started. These are located in the package tensorflow.examples.tutorials.

The dataset contains a total of 70,000 images, split up as follows:

- 55,000 training data
- 10,000 test data
- 5,000 validation data

Let's start by getting the data from tensorflow.

In [ ]:
# Import the MNIST input_data function from the tutorials.mnist package
from tensorflow.examples.tutorials.mnist import input_data

# Read in the data
# The paramter one_hot=True refers to the label which is categorical (1-10). 
# The paramter causes the label to be re-encoded as a 10 column vector.
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Let's now look at what type of object 'mnist' is:

In [ ]:
type(mnist)

You can see that it is a Tensorflow object of type Datasets. The training, test and validation data can be accessed
as attributes. 

Let's look at their data types. We can see they are Tensorflow Dataset (non-plural) data types.

In [ ]:
print( type(mnist.train) )
print( type(mnist.test) )
print( type(mnist.validation) )

## Inside the Dataset

Let's now look closer to what's inside the Tensorflow dataset object for the MNIST dataset. 

The images and corresponding labels are accessed by the attributes images and labels, respectively. How convenient, such a logical name for an attribute! 

Both the images and labels are a Numpy multi-dimensional array.

In [ ]:
type(mnist.train.images)

Let's check that the training data has the expected 55,000 images and labels.

In [ ]:
# Let's get the length (number of images) of the list of images.
print( len(mnist.train.images) )
print( len(mnist.train.labels) )

Let's now look at the contents of one of the images.

In [ ]:
mnist.train.images[0]


In [ ]:
mnist.train.images.reshape(-1,28,28).shape

The original black and white (bilevel) images from MNIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. The images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field.  - http://yann.lecun.com/exdb/mnist/

### Plotting an Image

Let's plot one of the images in the training set. To do so, we will use the plotting functions of the matplotlib package. Let's start by importing the package.

In [ ]:
import matplotlib.pyplot as plt

# This line is specific to python notebooks (not python). 
# It causes plots to automatically be rendered (displayed) without issuing a show command.
%matplotlib inline  

Let's now render (display) one of the images. 

<b>WAIT</b>, the images in the Tensorflow MNIST dataset are already flattened. That is, they are a single vector of 784 inputs. To display it, we need to reshape the vector back into a 28 x 28 pixel matrix. Then we can plot it.

In [ ]:
# Let's show that the shape of the image is already flatten (will output as 784)
print( mnist.train.images[1].shape )

# Let's now reshape it into a 28 x 28 matrix
image = mnist.train.images[1].reshape(28,28)
print( image.shape )

# Let's plot it now
plt.imshow( image )

### The Labels

The corresponding labels are a 2D matrix. Each row is for the corresponding image (that is index 0 in labels is for image at index 0 in images).

Each row has ten columns. The labels have been encoded as a one-hot encoding. In this case, if the image is a 3, then there is a 1 at index 3, and all the other columns are a zero.

Let's look at the label vector for the image above.

In [ ]:
# As you can see, there is a 1 at index 3 (fourth location, starting at 0)
mnist.train.labels[1]

## Convolutional Neural Network with Dropout Layer - Softmax Activation
We are going to build a simple convolutional neural network (CNN) with a dropout layer. The output layer from our neural network will be passed through a softmax activation function to produce our predictions of the digit.

In our Neural Network, we will have the following:
- An input convolution layer of 28x28 inputs and 32 28x28 outputs (filters)
- A linear recitifier activation function
- A max pooling layer of 32 (channels) 28x28 inputs and 32 (channels) 14x14 outputs
- A first hidden layer of 32x14x14 inputs and 256 outputs
- A dropout layer
- A linear recitifier activation function
- A second hidden layer of 256 inputs and 20 outputs
- A linear recitifier activation function
- An output layer of 20 inputs and 10 outputs
- A softmax activation function<br/>
INPUT CONVOLUTION LAYER => RELU => MAX POOLING => HIDDEN LAYER => DROPOUT LAYER => RELU => HIDDEN LAYER => RELU => OUTPUT LAYER => SOFTMAX

### Softmax

We will use the softmax layer to make our predictions. Each output from softmax will be a number between 0 and 1, representing a percent. That is, if the output for the node 3 is 0.8, then this means 80% prediction. We will choose the output with the highest percent when making a prediction.

Softmax is a mathematical function that takes a set of values, which may otherwise not add up to 1, and outputs a new set of numbers when totaled will add up to 1. That is, we use softmax() so that all our outputs for each image add up to 1 (100%).

Softmax will be our 'activation' function from the output layer.

## Building the Neural Network

Let's build our neural network now. We will do the following:

### Design, then Run

#### Design

    - Create the placeholder for the input data, dropout percentage, and learning rate
    - Design the layers
    - Set the optimizer
    
#### Run

      -- Initialize the Graph
      -- Set number of epochs
      -- Set batch size, learning rate
      -- Run the Graph with the Training Data to Train (learn) the Model
      -- Validate the Model with Test Data

### Input Vector and Output Vector and Hyperparameter Placeholders

For our first tensorflow step, we will setup a Tensorflow placeholders.

We have two placeholders we need to declare, one for the input vector (pixel image data) and one for the output vector (digit classifier).

For our input placeholder (which we call X), we have 784 features (pixels per image). For the output vector (which we call Y), we have have 10 classifiers (0 .. 9 digits). In both cases, we set the second dimension of our vector to None. The None is a placeholder for the number of samples we will feed into the neural network at run-time. We also know that our data is floating point values between 0 and 1, so we will set the data type to float32.

We will declare two more placeholders for setting some hyper-parameters, the percent to keep in the dropout layer (D) and the learning rate in the optimizer (L). Since both are scalar values, we will define their shape as a single value.

In [ ]:
# Let's first reset our graph, so our neural network components are all declared within the same graph
ops.reset_default_graph() 

In [ ]:
X = tf.placeholder(tf.float32, [None, 784]) # shape = [batch, number of pixels ]
Y = tf.placeholder(tf.float32, [None, 10])  # shape = [batch, number of labels ]
D = tf.placeholder(tf.float32, [])
L = tf.placeholder(tf.float32, [])

### INPUT (CONVOLUTION) LAYER

Let's now design our input convolution layer. For our convolutional layer, we will need a set of filers, weights for the filters and biases for the output. We will use 32 filters. Each filter will be 5 x 5 (pixels) in size and one channel (i.e., single plane) corresonding to grayscale image.

Each input filter will need a weight (which our model will learn during training). The weight is multipled against the value of the input (filter), which we symbolically represent as Wx. 

Each output from the layer will need a bias (which our model will learn during training). The bias is added to the result of the weight multipled by the filter (Wx + b).

Let's create two tensorflow variables for our weights and biases. The weights (which we call W) will need to be a 4D matrix. The first two dimensions are the filter size (5 x 5), then the number of channels, and then the number of outputs, which will be 32.

The bias will be a vector of size 32 (one for each output).

We need to initialize our weights and biases to some initial value. We will initialize the weights using a random value initializer (normalized distribution) and initialize the biases to 0.1.

In [ ]:
tf.set_random_seed(1)   # Set the same seed to get the same initialization as in this demo.

# The weights for the input (convolutional) layer
# 5x5 pixel filter, 1 channel, 32 outputs (filters)
W1 = tf.Variable(tf.truncated_normal([5, 5 , 1, 32], stddev=0.1))

# The bias for the output from the input (convolutional) layer
b1 = tf.Variable(tf.constant(0.1, shape=[32]))

Let's put it together into an input convolutional) layer. We will use the Tensorflow method tf.nn.conv2d() to apply the filters and the weights (our variable W1) against the inputs (our placeholder X), add in the bias (b1), and pass the output through a linear activation function.

- We need to reshape our flattened input data (X - which is our input placeholder) back into a 28x28 2D matric (bitmap) with one color channel - tf.reshape(X, [-1, 28, 28, 1])
- We will set our stride for the sliding filters to move one pixel at a time in each direction.
- We will set the padding when the filter moves past the edge of the bitmap to same.
- Add the bias to the 32 outputs from our convolution.
- Pass the outputs from the input (convolutional) layer through a RELU activation function

In [ ]:
# The first layer (2D Convolution)

Z1 = tf.nn.conv2d( input=tf.reshape(X, [-1, 28, 28, 1]),  
                   filter=W1,           
                   strides=[1,1,1,1],
                   padding='SAME') + b1

A1 = tf.nn.relu(Z1)


In [ ]:
# Let's look at the what the shape of the output tensor will be from the activation unit. 
# As you can see, it will be 28x28 pixels with 32 channels.
print(A1)

### MAX POOLING LAYER

The max pooling layer will have as input the output from the first layer, which is a 4D matrix (batch, height, width, channels), where the number of channels is 32. We will use a 2x2 pooling window over the one channel, with a stride of 2.

In [ ]:
# the second layer (max pooling)

Z2 = tf.nn.max_pool(A1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
# Let's look at the shape of the output tensor will be from the max pooling layer.
# As you can see, it has been downsampled to 14x14 pixels with 32 channels.
print(Z2)

### FIRST HIDDEN LAYER

The first hidden layer will have as inputs the flatten outputs from max pooling layer and 256 outputs. 

Let's start by flattening the output from the max pooling layer.

In [ ]:
F2 = tf.reshape(Z2, [-1, 14*14*32])  # Flatten each 14x14 pixel with 32 channels to single 1D vector
print(F2)
print(F2.get_shape()[1])

Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
# The return value from F2.get_shape() needs to be casted into an int.
W3 = tf.Variable(tf.truncated_normal([int(F2.get_shape()[1]), 256], stddev=0.1))

b3 = tf.Variable(tf.constant(0.1, shape=[256]))

Let's construct the first hidden layer

- Create a node that will multiply the weights (W3) against the outputs of the max pooling layer (F2)
- Create a node that adds the bias (b3) to the above node (F2 * W3).
- Pass the output of the hidden layer through a dropout layer
- Pass the outputs from the dropout layer through a RELU activation function

In [ ]:
# The third layer (first hidden layer)
Z3 = tf.add(tf.matmul(F2, W3), b3)

# Let's add the dropout layer to the output signal from the second layer
D3 = tf.nn.dropout(Z3, keep_prob=D)

# Let's add the activation function to the output signal from the dropout layer
A3 = tf.nn.relu(D3)

###### SECOND HIDDEN LAYER

The second hidden layer will have 256 inputs (outputs from first hidden layer) and 20 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W4 = tf.get_variable("W4", [256, 20], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b4 = tf.get_variable("b4", [1, 20], initializer=tf.zeros_initializer())

#### Let's construct the second hidden layer

- Create a node that will multiply the weights (W4) against the outputs of the first hidden layer (A3).
- Create a node that adds the bias (b4) to the above node (W4 * A3)
- Pass the outputs from the second hidden layer through a RELU activation function

In [ ]:
# The fourth layer (second hidden layer)
Z4 = tf.add(tf.matmul(A3, W4), b4) 

# Let's add the activation function to the output signal from the third layer
A4 = tf.nn.relu(Z4)

### OUTPUT LAYER

The output layer will have 20 inputs (outputs from the second hidden layer) and 10 outputs (one for each digit). Each input will need a weight and each output a bias (which we will train). The 10 outputs will be passed through a softmax activation function. 

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W5 = tf.get_variable("W5", [20, 10], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b5 = tf.get_variable("b5", [1, 10], initializer=tf.zeros_initializer())

Let's construct the output layer

- Create a node that will multiply the weights (W4) against the outputs of the second hidden layer (A3).
- Create a node that adds the bias to the above node (W4 * A3)
- Pass the outputs from the output layer through a SOFTMAX squashing function (done by the optimizer)

In [ ]:
# The fifth layer (output layer)
Z5 = tf.add(tf.matmul(A4, W5), b5) 

## OPTIMIZER

Now its time to design our optimizer. Let's start by designing our cost function. We will use the mean value of the softmax cross entropy between the predicted labels and actual labels. This is what we want to reduce on each batch.

In [ ]:
# to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z5, labels=Y))

Let's design our optimizer. This is the method that adjusts the values of the weights and biases, based on minizing the cost value during training.

We also need to set a learning rate. This is multiplied against the gradient calculation. It's used to prevent huge swings in setting weights which can result in either converging at a local (instead of global) optima, or not converging at all (infinite gradient). We will set the learning rate when we run the graph using the placeholder L.

In [ ]:
# The learning rate for Gradient Descent algorithm
#learning_rate = 0.5

optimizer = tf.train.GradientDescentOptimizer(L).minimize(cost)

### Run the Graph

We've built our Tensorflow graph for training our data. So, let's start training it.

First, we need to call Tensorflow's global_variables_initializer() method to initialize the variables we've defined. We will create this as another node, which will be the first node we run (evaluate) in our graph.

In [ ]:
init = tf.global_variables_initializer()

It's also a good idea to know how long your training takes, so let's import the time library.

In [ ]:
import time

Let's set our hyperparameters.

We need to set the number of epochs (that's how many times we run the training data through the neural network), and the batch size. The batch size is a small subset of the entire training set. We will be running a batch at a time per epoch. After each batch, then the cost is computed and backpropagated through the neural network.

In [ ]:
epochs = 20                                    # run a 20 epochs
batch_size = 200                               # for each epoch, train in batches of 200 images
number_of_images = mnist.train.labels.shape[0] # number of images in training data
batches = number_of_images // batch_size       # number of batches in an epoch
print("Number of batches:", batches)

keep_prob = 0.9                                # percent of outputs to keep in dropout layer
learning_rate = 0.5                            # the learning rate for graident descent

We are going to run the graph now!

We start by creating a tensorflow session (tf.Session()). Within the session we can run (evaluate) parts of the graph we designed.

We start by initializing the tensor variables we defined for the weights and biases.

We then run our training data through our neural network for the number of epochs we defined. For each epoch, we get a randomly shuffled batch from the training data and feed the batch (i.e. feed dictionary) into the neural network by running (evaluate)
the optimizer node in our graph.

Once we've trained the model, then we create some new nodes to calculate accuracy and evaluate against the training data.

In [ ]:
start = time.time()

with tf.Session() as sess:
    # Initialize the variables
    sess.run(init)
    
    # run our training data through the neural network for each epoch
    for epoch in range(epochs):
        
      epoch_cost = 0
      
      # Run the training data through the neural network
      for batch in range(batches):
          # Get a batch (random shuffled) from the training data
          batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      
          # Feed this batch through the neural network.
          _, batch_cost = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, D: keep_prob, L: learning_rate})
            
          epoch_cost += batch_cost
      
      print("Epoch: ", epoch, epoch_cost / batches, time.time() - start )
        
    end = time.time()
    
    print("Training Time:", end - start)
    
    # Test the Model
    
    # Let's select the highest percent from the softmax output per image as the prediction.
    prediction = tf.equal(tf.argmax(Z5), tf.argmax(Y))
    
    # Let's create another node for calculating the accuracy
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    # Now let's run our trainingt images through the model to calculate our accuracy during training
    # Note how we set the keep percent for the dropout rate to 1.0 (no dropout) when we are evaluating the accuracy.
    print ("Train Accuracy:", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels, D: 1.0}))
    
    # Now let's run our test images through the model to calculate our accuracy on the test data
    print ("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, D: 1.0}))

# Evaluate of Model

The last steps above are where our training and test data is ran through the model and produced how accurate our model was on the test data.

After training the model, we created a node for prediction. This node compares two vectors, our predicted labels and our actual labels. Each vector is 10 elements long with a 1 in the predicted/actual letter location. So we are comparing the vectors. If they match (prediction matches actual), then we have a TRUE; otherwise a FALSE. That's how we are going to get our accuracy percentage calculated.

Next, we create the node accuracy, which will take the average across the compared vectors.

We then run the accuracy node, feeding it first the training data and labels as X and Y. This will result in a forward feed pass through the network (no backward propagation). Then we feed it (forward feed ​the test data and labels as X and Y.

### Rerun with a different drop out rate
Let's change our dropout rate and see what happens. Let's go from dropping out 10% to 50% and run the cell above with the neural network again.

In [ ]:
# Let's drop out 75% of our nodes
learning_rate = 0.05

We see at 75% dropout that we start to plateau around 0.4 cost and are unable to converge. Yikes, we get only a 84% accuracy. 

Let's try it again with 50% dropout and rerun the above cell with the neural network again.

In [ ]:
# Let's drop out 50% of our nodes
keep_prob = 0.50

Okay, much better (~96%) accuracy. But we still got better with only a 10% dropout, so let's stick with that.

Let's see if we can get improvement with the learning rate. Let's try decreasing the learning rate by a magnitude and rerun the above cell with the neural network again.

In [ ]:
keep_prob = 0.90
learning_rate = 0.05

Aah, it's taking a little longer to converge, and we went down a tiny bit in accuracy. Let's do the opposite and set the learning rate closer to 1 and rerun the above cell with the neural network again.

In [ ]:
learning_rate = 0.9

Yikes! We are stuck on 2.4 cost on each plateau. Our learning rate is too high, it's causing us to bounce back and forth and never even start converging. This is an infinite gradient problem. 

Well, let's stick with our original hyper-parameters!

## You Have Now Completed TensorFlow's MNIST Example Extended to a CNN with a Dropout.

As you can see, we can affect converge and the accuracy by changing values for hyper-parameters like the drop out rate and learning rate.